In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px #visualize interactive image
import ace_tools_open as tools;

In [36]:
market_data = pd.read_csv("Data/market_data.csv", index_col='Date', parse_dates=True)
vix_data = pd.read_csv("Data/vix_data.csv", index_col='Date', parse_dates=True)
macro_data = pd.read_csv("Data/macro_monthly.csv", index_col='Date', parse_dates=True)
RIR_data = pd.read_csv("Data/real_interest_rate.csv", index_col='Date', parse_dates=True)

gpr_data = pd.read_excel("Data/data_gpr_export.xls", header = 0)
gpr_data = gpr_data.rename(columns={"month": "Date"})
gpr_data = gpr_data.set_index("Date")
gpr_selected = gpr_data[["GPR", "GPRT", "GPRA"]]

Resample GPR and macro_data to daily with ffill

In [37]:
gpr_daily = gpr_selected.resample('D').ffill()
macro_daily = macro_data.resample('D').ffill()

combined_data = market_data \
    .join(macro_daily, how='inner') \
    .join(RIR_data, how='inner') \
    .join(vix_data, how='inner') \
    .join(gpr_daily, how='inner')


tools.display_dataframe_to_user(name="Combined DataFrame", dataframe=combined_data)
combined_data.to_csv("Data/combined_data.csv")

Combined DataFrame


Loading ITables v2.4.2 from the internet... (need help?)


# 1. Understand - Explore Data

In [38]:
df = combined_data.copy()

In [39]:
df.info

<bound method DataFrame.info of             Crude_Oil  Gold_Futures    Gold_ETF     Silver  USD_Index  \
Date                                                                    
2003-01-02  31.850000    346.100006         NaN   4.812000        NaN   
2003-01-03  33.080002    351.200012         NaN   4.892000        NaN   
2003-01-06  32.099998    351.700012         NaN   4.899000        NaN   
2003-01-07  31.080000    347.299988         NaN   4.804000        NaN   
2003-01-08  30.559999    353.899994         NaN   4.856000        NaN   
...               ...           ...         ...        ...        ...   
2024-11-22  71.239998   2709.899902  249.839996  31.309000  29.020340   
2024-11-25  68.940002   2616.800049  242.479996  30.209999  28.819408   
2024-11-26  68.769997   2620.300049  242.949997  30.388000  28.857681   
2024-11-27  68.720001   2639.899902  243.490005  30.111000  28.608908   
2024-11-29  68.000000   2657.000000  245.589996  30.684999  28.561068   

                S&

Comment:
- All columns are numericals cols

In [40]:
df.columns

Index(['Crude_Oil', 'Gold_Futures', 'Gold_ETF', 'Silver', 'USD_Index',
       'S&P_500', 'NASDAQ', 'CPI', 'Interest_Rate', 'M2_Supply',
       'Unemployment', 'Treasury_Yield_10Y', 'Real_Interest_Rate', '^VIX',
       'GPR', 'GPRT', 'GPRA'],
      dtype='object')

In [29]:
# df.nunique()

In [ ]:
df.isnull().sum()

Crude_Oil               10
Gold_Futures            14
Gold_ETF               474
Silver                  13
USD_Index             1046
S&P_500                  0
NASDAQ                   0
CPI                      0
Interest_Rate            0
M2_Supply                0
Unemployment             0
Treasury_Yield_10Y       0
Real_Interest_Rate      41
^VIX                     0
GPR                      0
GPRT                     0
GPRA                     0
dtype: int64

## 2.Pre-processing